<a href="https://colab.research.google.com/github/SinSham/IMDB-Review-Sentiment-Analysis/blob/main/IMDB_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

**Importing Dependencies**

In [42]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

**Data Collection**

In [5]:
kaggle_dict = json.load(open('kaggle.json'))

In [6]:
kaggle_dict.keys()

dict_keys(['username', 'key'])

In [7]:
# setting up kaggle credentials as environment variables
os.environ['KAGGLE_USERNAME'] = kaggle_dict['username']
os.environ['KAGGLE_KEY'] = kaggle_dict['key']

In [8]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 97% 25.0M/25.7M [00:02<00:00, 21.3MB/s]
100% 25.7M/25.7M [00:02<00:00, 11.5MB/s]


In [9]:
#Unzipping the dataset file
with ZipFile('/content/imdb-dataset-of-50k-movie-reviews.zip', 'r') as zip_ref:
  zip_ref.extractall()

**Loading the Data**

In [11]:
data = pd.read_csv('/content/IMDB Dataset.csv')

In [12]:
data.shape

(50000, 2)

In [13]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [17]:
data['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [18]:
data.replace({'sentiment' : {'positive': 1, 'negative' : 0}}, inplace = True)

In [19]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [21]:
data['sentiment'].value_counts()

,count
sentiment,
1,25000
0,25000


In [22]:
#split data into training and test data

train_data, test_data = train_test_split(data, test_size = 0.2, random_state=42)

In [24]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


**Preprocessing the Data**

In [26]:
#Tokenize the text data

tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data['review'])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen=200)

In [27]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [28]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [29]:
y_train = train_data['sentiment']
y_test = test_data['sentiment']

In [30]:
print(y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


In [31]:
print(y_test)

33553    1
9427     1
199      0
12447    1
39489    0
        ..
28567    0
25079    1
18707    1
15200    0
5857     1
Name: sentiment, Length: 10000, dtype: int64


LSTM

In [46]:
# building the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [47]:
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [48]:
# compile the model

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

**Training the model**

In [49]:
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 145s 276ms/step - accuracy: 0.7379 - loss: 0.5190 - val_accuracy: 0.8439 - val_loss: 0.3755
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 205s 292ms/step - accuracy: 0.8394 - loss: 0.3876 - val_accuracy: 0.8376 - val_loss: 0.3734
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 191s 269ms/step - accuracy: 0.7679 - loss: 0.5179 - val_accuracy: 0.8583 - val_loss: 0.3362
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 136s 271ms/step - accuracy: 0.8795 - loss: 0.2960 - val_accuracy: 0.8708 - val_loss: 0.3012
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 137s 262ms/step - accuracy: 0.9083 - loss: 0.2330 - val_accuracy: 0.8740 - val_loss: 0.3035


**Model Evaluation**

In [50]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 27s 84ms/step - accuracy: 0.8799 - loss: 0.2899
Test Loss: 0.28849926590919495
Test Accuracy: 0.881600022315979


**Building a predictive system**

In [57]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [58]:
# example usage

test_review = "Movie was awesome"
sentiment = predict_sentiment(test_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
The sentiment of the review is: positive


In [59]:
# example usage

test_review = "Movie was not that good. It was mediocre"
sentiment = predict_sentiment(test_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
The sentiment of the review is: negative


In [60]:
# example usage

test_review = "Movie was boring"
sentiment = predict_sentiment(test_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
The sentiment of the review is: negative


In [62]:
model.save('IMDB_sentiment_analyzer__model.h5')

In [63]:
model.save('IMDB_sentiment_analyzer__model.keras')